<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task2/redrock_007_task2_tokenizer_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

### Dependencies

In [2]:
import importlib
import torch, transformers

if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0
  !pip install evaluate==0.4.2
  !pip install accelerate -U

if importlib.util.find_spec("wandb") is None:
  !pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl

If you've just installed `accelerate`, execute `Runtime > Restart session and run all` in the Colab UI menu above.

In [3]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.functional import F

import torchvision
from torchvision import datasets, transforms, models

from tqdm.auto import tqdm

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler, BertForMaskedLM, BertTokenizer
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset, DatasetDict

import torch.cuda.amp as amp # or import torch.cuda.amp as amp for PyTorch's native amp


import evaluate
import wandb

In [4]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task2"
    name = "redrock_006_task2_pretrain_wandb"

    # model
    base_model_name = "google-bert/bert-base-multilingual-uncased"
    tokenizer_name = "google-bert/bert-base-multilingual-uncased"
    mlm_probability = 0.15

    # training
    epochs = 1

    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    lr = 5e-05

    # dataset
    max_length = 256

    # T4: 32
    # L4: 64
    train_batch_size = 32

    seed=42
    train=True

In [5]:
wandb.login(key=userdata.get('wandb_token'))

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
# for wandb
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}

In [7]:
# classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)
# tokenizer = AutoTokenizer.from_pretrained(CFG.base_model_name)
brahmi_to_devanagari = {
    '𑀓': 'क', '𑀔': 'ख', '𑀕': 'ग', '𑀖': 'घ', '𑀗': 'ङ', '𑀘': 'च', '𑀙': 'छ',
    '𑀚': 'ज', '𑀛': 'झ', '𑀜': 'ञ', '𑀝': 'ट', '𑀞': 'ठ', '𑀟': 'ड', '𑀠': 'ढ',
    '𑀡': 'ण', '𑀢': 'त', '𑀣': 'थ', '𑀤': 'द', '𑀥': 'ध', '𑀦': 'न', '𑀧': 'प',
    '𑀨': 'फ', '𑀩': 'ब', '𑀪': 'भ', '𑀫': 'म', '𑀬': 'य', '𑀭': 'र', '𑀮': 'ल',
    '𑀯': 'व', '𑀰': 'श', '𑀱': 'ष', '𑀲': 'स', '𑀳': 'ह', '𑁦':'०', '𑁣': '90'
}

def transliterate_brahmi_to_devanagari(text):
    transliterated_text = ''
    for char in text:
        if char in brahmi_to_devanagari:
            transliterated_text += brahmi_to_devanagari[char]
        else:
            transliterated_text += char
    return transliterated_text

transliterate_dict = {
    'क': 'a', 'ख': 'b', 'ग': 'c', 'घ': 'd', 'ङ': 'e', 'च': 'f', 'छ': 'g',
    'ज': 'h', 'झ': 'i', 'ञ': 'j', 'ट': 'k', 'ठ': 'l', 'ड': 'm', 'ढ': 'n',
    'ण': 'o', 'त': 'p', 'थ': 'q', 'द': 'r', 'ध': 's', 'न': 't', 'प': 'u',
    'फ': 'v', 'ब': 'w', 'भ': 'x', 'म': 'y', 'य': 'z', 'र': 'A', 'ल': 'B',
    'व': 'C', 'श': 'D', 'ष': 'E', 'स': 'F', 'ह': 'G', '०': 'H', '90': 'I'
}

def transliterate_text(text):
    for key, value in transliterate_dict.items():
        text = text.replace(key, value)
    return text

def transliterate_to_latin(text):
    transliterated_text = ''
    for char in text:
        if char in brahmi_to_devanagari:
            transliterated_text += transliterate_text(brahmi_to_devanagari[char])
        else:
            transliterated_text += transliterate_text(char)
    return transliterated_text

def to_device(batch, device):
    output = {}
    for k, v in batch.items():
        try:
            output[k] = v.to(device)
        except:
            output[k] = v
    return output

In [8]:
raw_dataset = load_dataset('InternationalOlympiadAI/NLP_problem_raw', token=read_access_token)

Generating train split:   0%|          | 0/611245 [00:00<?, ? examples/s]

In [9]:
print(transliterate_to_latin(raw_dataset["train"]["text"][0]))

"""
file_name = "to_latin.txt"

for text in tqdm(raw_dataset["train"]["text"][:1000]):
    with open(file_name, "a") as f:
        f.write(transliterate_to_latin(text) + "\n")
"""

  yflfmfm rfgf pof fnFfmp qf wHmIuosH qIm lInfEf rtEf fBBHBHG lI nfnnfmpHEnfnnfmpHm qf ntutn lH jftlf f pplpm Efmp uflfnfpnfm rtnpm yfBpuuf lI Etxpm lEfoIopm yfBpuufC


'\nfile_name = "to_latin.txt"\n\nfor text in tqdm(raw_dataset["train"]["text"][:1000]):\n    with open(file_name, "a") as f:\n        f.write(transliterate_to_latin(text) + "\n")\n'

In [10]:
train_corpus = []

from joblib import Parallel, delayed

num_cores = 8


def process_text(x):
    train_corpus = []
    y = x
    if len(y) <= 1000:
        train_corpus.append(transliterate_to_latin(y))
    else:
        i = 0
        while len(y) >= 1000:
            ret = y[1000 * i : 1000 * (i + 1)]
            train_corpus.append(transliterate_to_latin(ret))
            y = y[1000 * (i + 1):]
            i += 1
        if len(y) > 0:
            train_corpus.append(transliterate_to_latin(y))
    return train_corpus

train_corpus_list = Parallel(n_jobs=-1)(delayed(process_text)(x) for x in tqdm(raw_dataset['train']["text"]))

train_corpus = [item for sublist in train_corpus_list for item in sublist]

print(train_corpus[:5])

  0%|          | 0/611245 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


['  yflfmfm rfgf pof fnFfmp qf wHmIuosH qIm lInfEf rtEf fBBHBHG lI nfnnfmpHEnfnnfmpHm qf ntutn lH jftlf f pplpm Efmp uflfnfpnfm rtnpm yfBpuuf lI Etxpm lEfoIopm yfBpuufC', ' f wfGfx EfGfmmpm IBonsppG gofm EfGfmGt Gtm Gfnt BfnnInpm ofnI Alt wfnp qf rpmfxH dC', 'xfonImq fmuyImo fBHIwyI qIlsHGp kfm yfpFH Gyp f xfmfx ld wf Efufm IluInf mf GyHlfxfx AqdA E lq nfot lulDB f wfxpm pnfqfm qfm lfGtEf mH mf lfFfFHm ofqf Bfnfxfp mf mffHxpofC', 'ofqqf flH lpEIm ufmufnfxfqtl ntutnpm qf of ufGI pplpm lfxlfxf EfuI iftoH ofmf qf pxxpm lpEIm qf of lfxnH GypC', 'xfGypm qfpqfpuI mf Gfntm ltqpm Gypwf lfnfx ofqqf f GyHlfxf uf luAut wpmp pIHFFpppHmu mf mffHxpof fm lpofGuf nfuGflfpppt f uC']


In [11]:
old_tokenizer = AutoTokenizer.from_pretrained(CFG.base_model_name)

tokenizer = old_tokenizer.train_new_from_iterator(train_corpus, old_tokenizer.vocab_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [12]:
print(transliterate_to_latin(raw_dataset["train"]["text"][0]))
print(tokenizer.tokenize(transliterate_to_latin(raw_dataset["train"]["text"][0])))

  yflfmfm rfgf pof fnFfmp qf wHmIuosH qIm lInfEf rtEf fBBHBHG lI nfnnfmpHEnfnnfmpHm qf ntutn lH jftlf f pplpm Efmp uflfnfpnfm rtnpm yfBpuuf lI Etxpm lEfoIopm yfBpuufC
['yflfmfm', 'rfgf', 'pof', 'fnffmp', 'qf', 'whmiuosh', 'qim', 'linfef', 'rtef', 'fbbhbhg', 'li', 'nfnnfmphenfnnfmphm', 'qf', 'ntutn', 'lh', 'jftlf', 'f', 'pplpm', 'efmp', 'uflfnfpnfm', 'rtnpm', 'yfbpuuf', 'li', 'etxpm', 'lefoiopm', 'yfbpuufc']


In [13]:
tokenizer.push_to_hub("ioai2024japan/redrock_007_task2_tokenizer", private=True, token=write_access_token)

CommitInfo(commit_url='https://huggingface.co/ioai2024japan/redrock_007_task2_tokenizer/commit/e5d06180e61e2ae826637231bd9599985e0840c6', commit_message='Upload tokenizer', commit_description='', oid='e5d06180e61e2ae826637231bd9599985e0840c6', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
def terminate_session():
    # Terminate this session

    from google.colab import runtime
    runtime.unassign()

In [15]:
terminate_session()

In [ ]:
import gc
import torch

def flush():
  gc.collect()
  torch.cuda.empty_cache()
flush()